In [1]:
from get_keys import wandb_key, hf_key

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth?tab=readme-ov-file#-installation-instructions).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

Features in the notebook:
1. Uses Maxime Labonne's [FineTome 100K](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset.
1. Convert ShareGPT to HuggingFace format via `standardize_sharegpt`
2. Train on Completions / Assistant only via `train_on_responses_only`
3. Unsloth now supports Torch 2.4, all TRL & Xformers versions & Python 3.12!

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

We included the token becuase we are using a gated model: unsloth/Llama-3.2-3B-Instruct

We chose this model becasuse it is extremely fast and efficient. Since the model will perform simple tasks, a chatbot, we prioritized speed over high-quality reasoning. It is a great balance between 1B and 3B, it equilibrates speed, resource usage, and credible quality



In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

## Create a model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_key, # use one if using gated models like meta-llama/Llama-2-7b-hf
)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model, # original model (the onde created before)
    r = 16, ## Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.11.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>

## If you get errors, change the normal multiturn format, there are multiple
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

To add more context, we merged two datasets: FineTome-100k and WizardLM_evol_instruct_70k. This second dataset will help the LLM model to increase conversation capability.

In [ ]:
from datasets import load_dataset

#dataset = load_dataset("mlabonne/FineTome-100k", split = "train")
#dataset = dataset.select(range(10000))

# Load FineTome-100k
finetome = load_dataset("mlabonne/FineTome-100k", split="train")
finetome = finetome.select(range(15000))

# Load WizardLM dataset to add context
wizardlm = load_dataset("WizardLMTeam/WizardLM_evol_instruct_70k", split="train")
wizardlm = wizardlm.select(range(15000))

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
#dataset = standardize_sharegpt(dataset) ## now we have a dataset
#dataset = dataset.map(formatting_prompts_func, batched = True,)

finetome = standardize_sharegpt(finetome) ## now we have a dataset
finetome = finetome.map(formatting_prompts_func, batched = True,)

In [ ]:
wizardlm = standardize_sharegpt(wizardlm) ## now we have a dataset

#Formatting the wizardlm dataset

def transform(example):
    return {
        "conversations": [
            {"role": "user", "content": example["instruction"]},
            {"role": "assistant", "content": example["output"]}
        ],
        "source": "wizardlm",
        "score": 0,  # default score
        "text": example["output"]
    }

def format_for_finetuning(example):
    # conversations is a list of dicts: [{'role': 'user', 'content': ...}, {'role': 'assistant', 'content': ...}]
    user_text = next(msg['content'] for msg in example['conversations'] if msg['role'] == 'user')
    assistant_text = next(msg['content'] for msg in example['conversations'] if msg['role'] == 'assistant')

    return {
        "text": (
            "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
            "Cutting Knowledge Date: December 2023\n"
            "Today Date: 26 July 2024\n\n"
            "<|start_header_id|>user<|end_header_id|>\n\n"
            f"{user_text}\n"
            "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
            f"{assistant_text}\n"
            "<|eot_id|>"
        )
    }
wizardlm = wizardlm.map(transform)
wizardlm = wizardlm.remove_columns(["output", "instruction"])

wizardlm = wizardlm.map(format_for_finetuning)

In [ ]:
print(finetome.column_names)
print(wizardlm.column_names)

['conversations', 'source', 'score', 'text']
['conversations', 'source', 'score', 'text']


In [ ]:
from datasets import concatenate_datasets
from datasets import Value

wizardlm = wizardlm.cast_column("score", Value("float32"))
finetome = finetome.cast_column("score", Value("float32"))

dataset = concatenate_datasets([wizardlm, finetome])

print(dataset.num_rows)

To have a more accurrate response, we will evaluate the output and feedbakc the LLM. To do this, we will split the dataset into training and test.



*--> The test dataset will be small to have faster iteration.*

In [ ]:
from datasets import DatasetDict

dataset = dataset.train_test_split(test_size=0.05, seed=42)

train_ds = dataset["train"]
eval_ds  = dataset["test"]

We look at how the conversations are structured for item 5:

In [ ]:
#import pandas as pd

#df = pd.read_parquet("hf://datasets/mlabonne/FineTome-100k/data/train-00000-of-00001.parquet")

In [ ]:
train_ds[5]["conversations"]

[{'content': 'Using a recursive algorithm, create a program that calculates the sum of the first 20 odd prime numbers and displays the result in hexadecimal and octal formats. In addition, provide a comparative analysis of the sum with the sum of the first 20 even prime numbers and the sum of the first 20 non-prime odd numbers. Finally, create a function that takes user input for the number of prime numbers they wish to sum and displays the result in binary format.',
  'role': 'user'},
 {'content': 'Here\'s the program to calculate the sum of the first 20 odd prime numbers using recursion:\n```python\ndef is_prime(n):\n    if n <= 1:\n        return False\n    for i in range(2, int(n**0.5)+1):\n        if n % i == 0:\n            return False\n    return True\ndef sum_odd_primes(n, i=3, count=0, total=0):\n    if count == n:\n        return total\n    if is_prime(i) and i % 2 == 1:\n        total += i\n        count += 1\n    return sum_odd_primes(n, i+1, count, total)\nodd_primes_sum 

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [ ]:
train_ds[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|start_header_id|>user<|end_header_id|>\n\nUsing a recursive algorithm, create a program that calculates the sum of the first 20 odd prime numbers and displays the result in hexadecimal and octal formats. In addition, provide a comparative analysis of the sum with the sum of the first 20 even prime numbers and the sum of the first 20 non-prime odd numbers. Finally, create a function that takes user input for the number of prime numbers they wish to sum and displays the result in binary format.\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHere\'s the program to calculate the sum of the first 20 odd prime numbers using recursion:\n```python\ndef is_prime(n):\n    if n <= 1:\n        return False\n    for i in range(2, int(n**0.5)+1):\n        if n % i == 0:\n            return False\n    return True\ndef sum_odd_primes(n, i=3, count=0, total=0

In [ ]:
# random sample from eval
import random
eval_ds[random.randint(0, len(eval_ds))]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nCan you design a lesson plan that integrates interactive activities and supports differentiated learning while exploring functions for 8th graders? The plan should encompass achievable goals, targeted formative evaluations, and a diverse set of 10 problems that require multi-step reasoning.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSure! Here's a lesson plan that aligns with your requirements:\r\n\r\nTitle: Exploring Functions through Interactive Activities\r\n\r\nGrade level: 8th grade\r\n\r\nSubject: Mathematics\r\n\r\nDuration: 90 minutes\r\n\r\nObjectives:\r\n\r\n- Students will understand the concept of functions and their representations through visual aids and real-life examples.\r\n- Students will be able to evaluate functions for a given input and understand the meaning of domain

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

To improve the performance of the LLM, we added callbacks

In [ ]:
from google.colab import drive
from transformers.trainer_callback import TrainerCallback

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_path = "/content/drive/MyDrive/Lab2"

In [ ]:
import os
from transformers.trainer_callback import TrainerCallback
import random

class checkpointcallback(TrainerCallback):
    def __init__(self, tokenizer, eval_prompts):
        self.best_train_loss = float("inf")
        self.tokenizer = tokenizer
        self.eval_prompts = eval_prompts

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and "loss" in logs:
            train_loss = logs.get("loss")
            if train_loss is not None and train_loss < self.best_train_loss:
                self.best_train_loss = train_loss

                lr = args.learning_rate
                batchsize = args.per_device_train_batch_size
                nsteps = state.global_step

                # Build save path
                save_path = os.path.join(
                    drive_path,
                    f"weights_lr={lr}_batchsize={batchsize}_nsteps={nsteps}_train_loss={train_loss:.2f}.pt"
                )

                model_to_save = kwargs['model'] # Changed from state.model
                model.save_pretrained(save_path)
                print(f"Saved best weights to {save_path}")

    def on_epoch_end(self, args, state, control, **kwargs):
        model = kwargs["model"]
        model.eval()

        # Select a random prompt
        prompt = random.choice(self.eval_prompts)
        print(f"\n[Random Eval Prompt] Prompt:\n{prompt}")

        # Tokenize and generate
        inputs = self.tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=200,
                temperature=0.3,
                do_sample=False,   # deterministic
            )

        response = self.tokenizer.decode(output[0], skip_special_tokens=True)
        print("\n[Model Response]")
        print(response)
        print("\n" + "-"*60 + "\n")


To improve the performance of the model, the following hyperparameters were modified:

*   Learning rate decreased 1e-1 to avoid overfitting in early steps and instability
*   Increased warmup to 50 to avoid hallucinations and divergence early
*   Reduced the maximum steps to prevent overfit, hallucinations and the model to lose general abilities
*   Deleted the decay weight as it can degrade quality, slower convergence and casue hallucination
*   Increased per_device_train_batch_size to 8 in order to speed up training
*   To reach an effective batch size = 16 (due to Google colab having 16 GB og GPU), gradient_accumulation_steps was set to 4
*   logging_steps incremented to 10 to track loss without spamming
*   Added gradient_checkpoint to save strategically selected activations and re-computed only a fraction of the activations for the gradients. This will help to prevent saving all activations from the forward pass.

In [ ]:
import wandb

# Login to WandB
wandb.login(key=wandb_key)

wandb.init(
    project="just-trying",
    name="llama3.2-3b-q4k-finetome-wizardlm",
    entity="anakaren-marqueze-kth-royal-institute-of-technology"
)

# Prompts used to evaluate
eval_prompts = [
    "Explain how photosynthesis works",
    "What is the difference between an orange and an apple?",
    "Write a short definition of physics",
    "Explain the importance of astrophysics",
]

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
import itertools


# Define the hyperparameter grid
param_grid = {
    "learning_rate": [1e-5, 2e-5],
    "per_device_train_batch_size": [2, 4],
    "gradient_accumulation_steps": [2, 4],
    "warmup_steps": [25, 50],
}

# Iterate over all combinations
for lr, batch_size, grad_acc, warmup in itertools.product(
    param_grid["learning_rate"],
    param_grid["per_device_train_batch_size"],
    param_grid["gradient_accumulation_steps"],
    param_grid["warmup_steps"]
):
    print(f"\n=== Training with lr={lr}, batch_size={batch_size}, grad_acc={grad_acc}, warmup={warmup} ===\n")

    # Update output folder per combination
    output_dir = f"outputs/lr{lr}_bs{batch_size}_ga{grad_acc}_warmup{warmup}"

    # Define training args
    args = TrainingArguments(
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=grad_acc,
        warmup_steps=warmup,
        max_steps=40,  # keep small for fast test runs; increase for full training
        learning_rate=lr,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=20,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir=output_dir,
        report_to="wandb",
    )

    # Initialize trainer
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
        dataset_num_proc=2,
        packing=False,
        args=args,
    )

    # Train
    trainer.train()



=== Training with lr=1e-05, batch_size=2, grad_acc=2, warmup=25 ===



Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/28500 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1500 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 28,500 | Num Epochs = 1 | Total steps = 40
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Step,Training Loss
20,1.373100
40,1.292300


...

Step,Training Loss
20,0.911700
40,0.944900


train/epoch,▁██
train/global_step,▁██
train/grad_norm,▁█
train/learning_rate,█▁
train/loss,▁█
total_flos,4147776171442176.0
train/epoch,0.01123
train/global_step,40
train/grad_norm,0.28925
train/learning_rate,0.0
train/loss,0.9449



=== Training with lr=2e-05, batch_size=4, grad_acc=2, warmup=50 ===



The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 28,500 | Num Epochs = 1 | Total steps = 40
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Step,Training Loss


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

## SFTTrainer --> Pytorch
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    callbacks=[checkpointcallback(tokenizer, eval_prompts)], #added callbacks
    eval_dataset = eval_ds, ## add the evaluation dataset
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.

    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        gradient_checkpointing = True,
        warmup_steps = 50,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 40,
        learning_rate = 2e-5, # Decreased 1e-1
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit", # This optimizer keeps the full state and quantizes it, has low precision but saves memory
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
        eval_strategy = "steps",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/28500 [00:00<?, ? examples/s]

TimeoutError: 

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|start_header_id|>user<|end_header_id|>\n\nUsing a recursive algorithm, create a program that calculates the sum of the first 20 odd prime numbers and displays the result in hexadecimal and octal formats. In addition, provide a comparative analysis of the sum with the sum of the first 20 even prime numbers and the sum of the first 20 non-prime odd numbers. Finally, create a function that takes user input for the number of prime numbers they wish to sum and displays the result in binary format.\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHere\'s the program to calculate the sum of the first 20 odd prime numbers using recursion:\n```python\ndef is_prime(n):\n    if n <= 1:\n        return False\n    for i in range(2, int(n**0.5)+1):\n        if n % i == 0:\n            return False\n    return True\ndef sum_odd_primes(n, i=3,

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                Here\'s the program to calculate the sum of the first 20 odd prime numbers using recursion:\n```python\ndef is_prime(n):\n    if n <= 1:\n        return False\n    for i in range(2, int(n**0.5)+1):\n        if n % i == 0:\n            return False\n    return True\ndef sum_odd_primes(n, i=3, count=0, total=0):\n    if count == n:\n        return total\n    if is_prime(i) and i % 2 == 1:\n        total += i\n        count += 1\n    return sum_odd_primes(n, i+1, count, total)\nodd_primes_sum = sum_odd_primes(20)\nhex_sum = hex(odd_primes_sum)\noct_sum = oct(odd_primes_sum)\nprint(f"The sum of the first 20 odd prime numbers is {odd_primes_sum}")\nprint(f"The sum in hexadecimal format is {hex_sum}")\nprint(f"The sum in octal format is {oct_sum}")\n```\nThe output will be:\n```\nThe sum of the first 20 odd prime numbers is 639\nThe sum in hexadecimal format is 0x2

We can see the System and Instruction prompts are successfully masked!

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.506 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 28,500 | Num Epochs = 1 | Total steps = 40
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Step,Training Loss,Validation Loss
10,0.832300,0.824035
20,0.823000,0.813952
30,0.796900,0.798576
40,0.864400,0.782832


Saved best weights to /content/drive/MyDrive/Lab2/weights_lr=2e-05_batchsize=4_nsteps=10_train_loss=0.83.pt
Saved best weights to /content/drive/MyDrive/Lab2/weights_lr=2e-05_batchsize=4_nsteps=20_train_loss=0.82.pt
Saved best weights to /content/drive/MyDrive/Lab2/weights_lr=2e-05_batchsize=4_nsteps=30_train_loss=0.80.pt

[Random Eval Prompt] Prompt:
Explain how photosynthesis works

[Model Response]
Explain how photosynthesis works
Photosynthesis is the process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of organic compounds, such as glucose. This process is essential for life on Earth, as it provides the energy and organic compounds needed to support the food chain.
The overall equation for photosynthesis is:
6 CO2 + 6 H2O + light energy → C6H12O6 (glucose) + 6 O2
In this equation, carbon dioxide and water are converted into glucose and oxygen. The light energy is absorbed by pigments such as chlorophyll, which is pres

eval/loss,█▆▄▁
eval/runtime,▅█▂▁
eval/samples_per_second,▄▁▆█
eval/steps_per_second,▁▁██
train/epoch,▁▂▂▄▄▆▆███
train/global_step,▁▂▂▄▄▆▆███
train/grad_norm,▅█▅▅▁
train/learning_rate,▁▂▄▆█
train/loss,▄▅▄▁█
eval/loss,0.78283
eval/runtime,604.6453


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2975.0112 seconds used for training.
49.58 minutes used for training.
Peak reserved memory = 7.967 GB.
Peak reserved memory for training = 0.461 GB.
Peak reserved memory % of max memory = 54.047 %.
Peak reserved memory for training % of max memory = 3.127 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In this section, we made this changes:

*   Reduce temperature to 0.2 to prevent hallucinations



In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]

## modify these hyperparameters too
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 0.2, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContinue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe Fibonacci sequence continues as follows:\n\n1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 418']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Fibonacci sequence continues with the following terms:

- 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 985, 1597<|eot_id|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.push_to_hub("Ana8413/model_02dec_lora", token = hf_key) # Online saving --> write where the hugging face registry is located
tokenizer.push_to_hub("Ana8413/model_02dec_lora", token = hf_key) # Online saving

README.md:   0%|          | 0.00/612 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 45.8kB / 97.3MB            

Saved model to https://huggingface.co/Ana8413/model_02dec_lora


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpnumsxd_o/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of Singapur."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("new_complete_model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("Ana8413/new_complete_model", tokenizer, save_method = "merged_16bit", token = hf_key)

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("Ana8413/model", tokenizer, save_method = "merged_4bit", token = hf_key)

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("Ana8413/model", tokenizer, save_method = "lora", token = hf_key)

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("Ana8413/model", tokenizer, token =hf_key)

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("Ana8413/model", tokenizer, quantization_method = "f16", token = hf_key)

# Save to q4_k_m GGUF
if True: model.save_pretrained_gguf("model_30nov", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("Ana8413/model_30nov", tokenizer, quantization_method = "q4_k_m", token = hf_key)

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "Ana8413/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = hf_key, # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>